In [ ]:
import pandas as pd

df = pd.read_csv("https://api.featurelabs.com/datasets/online-retail-logs-2018-08-28.csv")
df['order_product_id'] = range(df.shape[0])
df.head(5)

As you can see, this is a dataframe containing several different data types, including dates, categorical values, numeric values, and natural language descriptions. Next, initialize Woodwork on this DataFrame.

## Initializing Woodwork on a DataFrame
Importing Woodwork creates a special namespace on your DataFrames, `DataFrame.ww`, that can be used to set or update the typing information for the DataFrame. As long as Woodwork has been imported, initializing Woodwork on a DataFrame is as simple as calling `.ww.init()` on the DataFrame of interest. An optional name parameter can be specified to label the data.

In [ ]:
import woodwork as ww

df.ww.init(name="retail")
df.ww

Using just this simple call, Woodwork was able to infer the logical types present in the data by analyzing the DataFrame dtypes as well as the information contained in the columns. In addition, Woodwork also added semantic tags to some of the columns based on the logical types that were inferred.

All Woodwork methods and properties can be accessed through the `ww` namespace on the DataFrame. DataFrame methods called from the Woodwork namespace will be passed to the DataFrame, and whenever possible, Woodwork will be initialized on the returned object, assuming it is a Series or a DataFrame.

As an example, use the `head` method to create a new DataFrame containing the first 5 rows of the original data, with Woodwork typing information retained.

In [ ]:
head_df = df.ww.head(5)
head_df.ww

In [ ]:
head_df

## Updating Logical Types
If the initial inference was not to our liking, the logical type can be changed to a more appropriate value. Let's change some of the columns to a different logical type to illustrate this process. In this case, set the logical type for the `order_product_id` and `country` columns to be `Categorical` and set `customer_name` to have a logical type of `PersonFullName`.

In [ ]:
df.ww.set_types(logical_types={
    'customer_name': 'PersonFullName',
    'country': 'Categorical',
    'order_product_id': 'Categorical'
})
df.ww.types

Inspect the information in the `types` output. There, you can see that the Logical type for the three columns has been updated with the logical types you specified.

## Selecting Columns

Now that you've prepared logical types, you can select a subset of the columns based on their logical types. Select only the columns that have a logical type of `Integer` or `Double`.

In [ ]:
numeric_df = df.ww.select(['Integer', 'Double'])
numeric_df.ww

This selection process has returned a new Woodwork DataFrame containing only the columns that match the logical types you specified. After you have selected the columns you want, you can use the DataFrame containing just those columns as you normally would for any additional analysis.

In [ ]:
numeric_df

## Adding Semantic Tags

Next, let’s add semantic tags to some of the columns. Add the tag of `product_details` to the `description` column, and tag the `total` column with `currency`.

In [ ]:
df.ww.set_types(semantic_tags={'description':'product_details', 'total': 'currency'})
df.ww

Select columns based on a semantic tag. Only select the columns tagged with `category`.

In [ ]:
category_df = df.ww.select('category')
category_df.ww

Select columns using multiple semantic tags or a mixture of semantic tags and logical types.

In [ ]:
category_numeric_df = df.ww.select(['numeric', 'category'])
category_numeric_df.ww

In [ ]:
mixed_df = df.ww.select(['Boolean', 'product_details'])
mixed_df.ww

To select an individual column, specify the column name. Woodwork will be initialized on the returned Series and you can use the Series for additional analysis as needed.

In [ ]:
total = df.ww['total']
total.ww

In [ ]:
total

Select multiple columns by supplying a list of column names.

In [ ]:
multiple_cols_df = df.ww[['product_id', 'total', 'unit_price']]
multiple_cols_df.ww

## Removing Semantic Tags
Remove specific semantic tags from a column if they are no longer needed. In this example, remove the `product_details` tag from the `description` column.

In [ ]:
df.ww.remove_semantic_tags({'description':'product_details'})
df.ww

Notice how the ``product_details`` tag has been removed from the ``description`` column. If you want to remove all user-added semantic tags from all columns, you can do that, too.

In [ ]:
df.ww.reset_semantic_tags()
df.ww

## Set Index and Time Index
At any point, you can designate certain columns as the Woodwork `index` or `time_index` with the methods [set_index](generated/woodwork.table_accessor.WoodworkTableAccessor.set_index.rst) and [set_time_index](generated/woodwork.table_schema.TableSchema.set_time_index.rst). These methods can be used to assign these columns for the first time or to change the column being used as the index or time index.

Index and time index columns contain `index` and `time_index` semantic tags, respectively.

In [ ]:
df.ww.set_index('order_product_id')
df.ww.index

In [ ]:
df.ww.set_time_index('order_date')
df.ww.time_index

In [ ]:
df.ww

In [ ]:
series = pd.Series([1, 2, 3], dtype='int64')
series.ww.init(logical_type='Integer')
series.ww

In the example above, we specified the `Integer` LogicalType for the Series. Because `Integer` has a physical type of `int64` and this matches the dtype used to create the Series, no Series dtype conversion was needed and the initialization succeeds.

In cases where the LogicalType requires the Series dtype to change, a helper function `ww.init_series` must be used. This function will return a new Series object with Woodwork initialized and the dtype of the series changed to match the physical type of the LogicalType.

To demonstrate this case, first create a Series, with a `string` dtype. Then, initialize a Woodwork Series with a `Categorical` logical type using the `init_series` function. Because `Categorical` uses a physical type of `category`, the dtype of the Series must be changed, and that is why we must use the `init_series` function here.

The series that is returned will have Woodwork initialized with the LogicalType set to `Categorical` as expected, with the expected dtype of `category`.

In [ ]:
string_series = pd.Series(['a', 'b', 'a'], dtype='string')
ww_series = ww.init_series(string_series, logical_type='Categorical')
ww_series.ww

As with DataFrames, Woodwork provides several methods that can be used to update or change the typing information associated with the series. As an example, add a new semantic tag to the series.

In [ ]:
series.ww.add_semantic_tags('new_tag')
series.ww

As you can see from the output above, the specified tag has been added to the semantic tags for the series.

You can also access Series properties methods through the Woodwork namespace. When possible, Woodwork typing information will be retained on the value returned. As an example, you can access the Series `shape` property through Woodwork.

In [ ]:
series.ww.shape

You can also call Series methods such as `sample`. In this case, Woodwork typing information is retained on the Series returned by the `sample` method.

In [ ]:
sample_series = series.ww.sample(2)
sample_series.ww

In [ ]:
sample_series

## List Logical Types
Retrieve all the Logical Types present in Woodwork. These can be useful for understanding the Logical Types, as well as how they are interpreted.

In [ ]:
from woodwork.type_sys.utils import list_logical_types

list_logical_types()